In [ ]:
!curl https://ollama.ai/install.sh | sh
!command -v systemctl >/dev/null && sudo systemctl stop ollama

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100  8354    0  8354    0     0   9370      0 --:--:-- --:--:-- --:--:--  9365
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 0.0.0.0:11434.
>>> Install complete. Run "ollama" from the command line.
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [ ]:
!ollama serve > server.log 2>&1 &

In [ ]:
import os
os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"
os.environ["LD_LIBRARY_PATH"] = "/usr/lib64-nvidia"

In [ ]:
!ollama run mistral > server_log.log 2>&1 &

In [ ]:
!pip install litellm[proxy]

In [ ]:
from litellm import completion

response = completion(
    model="ollama/mistral",
    messages=[{ "content": "respond in 20 words. who are you?","role": "user"}],
    api_base="http://127.0.0.1:11434",
    stream=True
)
print(response)
for chunk in response:
    print(chunk['choices'][0]['delta'])

<generator object ollama_completion_stream at 0x7f9e7a763990>
Delta(content='I', role='assistant')
Delta(content="'", role=None)
Delta(content='m', role=None)
Delta(content=' an', role=None)
Delta(content=' artificial', role=None)
Delta(content=' intelligence', role=None)
Delta(content=' designed', role=None)
Delta(content=' to', role=None)
Delta(content=' assist', role=None)
Delta(content=' and', role=None)
Delta(content=' communicate', role=None)
Delta(content=' with', role=None)
Delta(content=' people', role=None)
Delta(content='.', role=None)
Delta(content=' Your', role=None)
Delta(content=' query', role=None)
Delta(content=' was', role=None)
Delta(content=' very', role=None)
Delta(content=' brief', role=None)
Delta(content=',', role=None)
Delta(content=' so', role=None)
Delta(content=' my', role=None)
Delta(content=' response', role=None)
Delta(content=' is', role=None)
Delta(content=' also', role=None)
Delta(content=' kept', role=None)
Delta(content=' short', role=None)
Delta(con

In [ ]:
!pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 7.9 MB/s eta 0:00:00


In [ ]:
!litellm --model ollama/mistral --api_base "http://127.0.0.1:11434" --debug > server_log_2.log 2>&1 &

In [ ]:
from autogen import AssistantAgent, GroupChatManager, UserProxyAgent
from autogen.agentchat import GroupChat

# Configuration for connecting to a model server
config_list = [
    {
        "model": "ollama/mistral",
        "base_url": "http://localhost:8000",  # litellm compatible endpoint
        "api_key": "NULL",  # placeholder
    }
]
llm_config = {"config_list": config_list}

# Setup Agents
researcher = AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
    system_message="""Researcher. Extract and rank keywords based on their relevance and meaning from given texts. Specialized in ontology analysis and biology.""",
)

pubmed_master = AssistantAgent(
    name="PubMed Master",
    llm_config=llm_config,
    system_message="""PubMed Master. Your role is to generate and optimize PubMed queries by intelligently using logical operators like AND and OR. You are specialized in crafting effective search queries for biomedical literature. Your task involves creating queries in a structured 'AND/OR' format, where you will pair each extracted keyword with every other keyword in a combinatory fashion. For each pair of keywords, formulate a query using the format: (Keyword1 AND Keyword2) OR Keyword3. This approach ensures a comprehensive exploration of the research topic by covering various keyword combinations. Your goal is to generate a list of queries that effectively capture the diverse aspects of the research topic, facilitating the retrieval of the most relevant and informative articles from PubMed.""",
)


admin = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Reviews and approves the plan and query created by the agents.",
    llm_config=llm_config,
    code_execution_config=False,
)

# Group Chat Setup
groupchat = GroupChat(
    agents=[admin, researcher, pubmed_master],
    messages=[],
    max_round=50,
)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Abstract to be analyzed
abstract = "Most Staphylococcus aureus strains can grow as a multicellular biofilm, a phenotype of utmost importance to clinical infections such as endocarditis, osteomyelitis, and implanted medical device infection. As biofilms are inherently more tolerant to the host immune system and antibiotics, understanding the S. aureus genes and regulatory circuits that contribute to biofilm development is an active and ongoing field of research. This chapter details a high-throughput and standardized way to grow S. aureus biofilms using a classical microtiter plate assay. Biofilms can be quantified using crystal violet or by confocal microscopy imaging and COMSTAT analysis."

# Initiating the Chat with the Abstract
admin.initiate_chat(
    manager,
    message=f"Extract keywords from the following abstract and create a set of PubMed queries using the 'AND/OR' format: {abstract}",
)


Admin (to chat_manager):

Extract keywords from the following abstract and create a set of PubMed queries using the 'AND/OR' format: Most Staphylococcus aureus strains can grow as a multicellular biofilm, a phenotype of utmost importance to clinical infections such as endocarditis, osteomyelitis, and implanted medical device infection. As biofilms are inherently more tolerant to the host immune system and antibiotics, understanding the S. aureus genes and regulatory circuits that contribute to biofilm development is an active and ongoing field of research. This chapter details a high-throughput and standardized way to grow S. aureus biofilms using a classical microtiter plate assay. Biofilms can be quantified using crystal violet or by confocal microscopy imaging and COMSTAT analysis.

--------------------------------------------------------------------------------
PubMed Master (to chat_manager):

 1. ("Staphylococcus aureus" AND "biofilm") OR ("S. aureus" AND "multicellular biofilm")